In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import scipy.sparse as sp
import random

2024-07-11 15:55:03.571267: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-11 15:55:05.510048: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 15:55:05.510101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 15:55:05.678694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-11 15:55:06.020542: I tensorflow/core/platform/cpu_feature_guar

In [2]:
drugs_cell_lines_ic50_df = pd.read_csv("..//data//drugs_cell_lines_ic50.csv")

In [3]:
drugs_cell_lines_ic50_df.head()

,drug_id,Cancer_Cell_Line,IC50
0,1001,ACH-002137,7.258918
1,1004,ACH-002137,-3.802467
2,1005,ACH-002137,4.146364
3,1006,ACH-002137,3.171367
4,1007,ACH-002137,-4.959442


In [4]:
drugs_cell_lines_ic50_df.shape

(208734, 3)

In [5]:
pubchem_drugs_smiles_df = pd.read_csv('..//data//drugs_smile_strings.csv')

In [6]:
pubchem_drugs_smiles_df.head()

,drug_id,Smiles
0,1242,COc1cc(O)c2c(c1)C=CCC(O)C(O)C(=O)C=CCC(C)OC2=O
1,179,O=c1[nH]cc(F)c(=O)[nH]1
2,86,Cc1[nH]nc2ccc(-c3cncc(OCC(N)Cc4c[nH]c5ccccc45)...
3,55,COc1cc(-c2nn(C3CCC(N4CCN(C(C)=O)CC4)CC3)c3ncnc...
4,1001,NC(=O)c1ncn(C2OC(COP(=O)(O)O)C(O)C2O)c1N


In [7]:
pubchem_drugs_smiles_df.shape

(238, 2)

In [8]:
drugs_smiles_cell_lines_ic50_df = pd.merge(drugs_cell_lines_ic50_df, pubchem_drugs_smiles_df, 
                                             on = "drug_id")

In [9]:
drugs_smiles_cell_lines_ic50_df.head()

,drug_id,Cancer_Cell_Line,IC50,Smiles
0,1001,ACH-002137,7.258918,NC(=O)c1ncn(C2OC(COP(=O)(O)O)C(O)C2O)c1N
1,1004,ACH-002137,-3.802467,CCC1(O)CC2CN(CCc3c([nH]c4ccccc34)C(C(=O)OC)(c3...
2,1005,ACH-002137,4.146364,N.N.[Cl-].[Cl-].[Pt+2]
3,1006,ACH-002137,3.171367,Nc1ccn(C2OC(CO)C(O)C2O)c(=O)n1
4,1007,ACH-002137,-4.959442,CC(=O)OC12COC1CC(O)C1(C)C(=O)C(O)C3=C(C)C(OC(=...


In [10]:
drugs_smiles_cell_lines_ic50_df.shape

(186789, 4)

In [11]:
drugs_smiles_cell_lines_ic50_df = drugs_smiles_cell_lines_ic50_df[["drug_id", "Cancer_Cell_Line", "Smiles", "IC50"]]

In [12]:
drugs_smiles_cell_lines_ic50_df.dtypes

drug_id               int64
Cancer_Cell_Line     object
Smiles               object
IC50                float64
dtype: object

In [13]:
drugs_smiles_cell_lines_ic50_df["drug_id"] = drugs_smiles_cell_lines_ic50_df["drug_id"].astype(object)

In [14]:
drugs_smiles_cell_lines_ic50_df.shape

(186789, 4)

In [15]:
with open("..//data//drug_gcn_features.pickle", "rb") as f:
    dict_features = pickle.load(f)

In [16]:
with open("..//data//drug_gcn_normalized_adj_mats.pickle", "rb") as f:
    dict_normalized_adj_mats = pickle.load(f)

In [17]:
dualgcn_train = pd.read_csv("..//data//DualGCN_Embedding_train.csv")

In [18]:
dualgcn_train.shape

(69214, 388)

In [19]:
dualgcn_train.head()

,Cell_Line,Drug_ID,Target,TCGA_Type,DrugEmbedding_Dim0,DrugEmbedding_Dim1,DrugEmbedding_Dim2,DrugEmbedding_Dim3,DrugEmbedding_Dim4,DrugEmbedding_Dim5,...,CellEmbedding_Dim246,CellEmbedding_Dim247,CellEmbedding_Dim248,CellEmbedding_Dim249,CellEmbedding_Dim250,CellEmbedding_Dim251,CellEmbedding_Dim252,CellEmbedding_Dim253,CellEmbedding_Dim254,CellEmbedding_Dim255
0,ACH-000070,176870,0.693305,ALL,0.128356,0.011836,0.243534,0.137116,-0.13091,-0.240304,...,-0.176026,0.054300,0.662075,-0.400942,-0.207140,-0.064926,-0.206338,-0.088848,-0.184410,-0.059895
1,ACH-000105,176870,1.738326,ALL,0.128356,0.011836,0.243534,0.137116,-0.13091,-0.240304,...,-0.121042,0.124696,-0.560812,-0.504415,-0.134911,-0.005806,-0.135737,-0.018490,-0.109957,-0.004947
2,ACH-000981,176870,2.838543,ALL,0.128356,0.011836,0.243534,0.137116,-0.13091,-0.240304,...,-0.128809,0.110884,-0.560812,-0.504415,-0.151752,-0.017094,-0.150296,-0.034415,-0.125914,-0.020937
3,ACH-000061,176870,3.880844,ALL,0.128356,0.011836,0.243534,0.137116,-0.13091,-0.240304,...,-0.172548,0.060355,0.710201,0.061378,-0.201455,-0.061008,-0.201532,-0.083990,-0.176958,-0.052960
4,ACH-001106,176870,2.112241,ALL,0.128356,0.011836,0.243534,0.137116,-0.13091,-0.240304,...,-0.176026,0.054300,-0.560812,-0.495085,-0.207140,-0.064926,-0.206338,-0.088848,-0.184410,-0.059895


In [20]:
dualgcn_test = pd.read_csv("..//data//DualGCN_Embedding_test.csv")

In [21]:
dualgcn_test.shape

(17316, 388)

In [22]:
pubchem_to_drugs_df = pd.read_csv('..//data//1.Drug_listMon Jun 24 09_00_55 2019.csv')

In [23]:
pubchem_to_drugs_df.head()

,drug_id,Name,Synonyms,Targets,Target pathway,PubCHEM,Sample Size,Count
0,1242,(5Z)-7-Oxozeaenol,"5Z-7-Oxozeaenol, LL-Z1640-2",TAK1,"Other, kinases",9863776,945,266
1,179,5-Fluorouracil,5-FU,Antimetabolite (DNA & RNA),Other,3385,968,266
2,86,A-443654,KIN001-139,"AKT1, AKT2, AKT3",PI3K/MTOR signaling,10172943,425,266
3,55,A-770041,KIN001-111,"LCK, FYN","Other, kinases",9549184,426,266
4,1001,AICA Ribonucleotide,"AICAR, N1-(b-D-Ribofuranosyl)-5-aminoimidazole...",AMPK agonist,Metabolism,65110,872,266


In [24]:
pubchem_to_drugs_df.shape

(266, 8)

In [25]:
pubchem_to_drugs_df = pubchem_to_drugs_df[["drug_id", "PubCHEM"]]

In [26]:
pubchem_to_drugs_df.dtypes

drug_id     int64
PubCHEM    object
dtype: object

In [27]:
pubchem_to_drugs_df["PubCHEM"] = [val if str(val).isdigit() else np.nan for val in pubchem_to_drugs_df["PubCHEM"] ]

In [28]:
pubchem_to_drugs_df = pubchem_to_drugs_df.dropna()

In [29]:
pubchem_to_drugs_df.dtypes

drug_id     int64
PubCHEM    object
dtype: object

In [30]:
pubchem_to_drugs_df["drug_id"] = pubchem_to_drugs_df["drug_id"].astype(str)

In [31]:
pubchem_to_drugs_df.head()

,drug_id,PubCHEM
0,1242,9863776
1,179,3385
2,86,10172943
3,55,9549184
4,1001,65110


In [32]:
pubchem_to_drugs_df.shape

(238, 2)

In [33]:
drugs_smiles_cell_lines_ic50_df.head()

,drug_id,Cancer_Cell_Line,Smiles,IC50
0,1001,ACH-002137,NC(=O)c1ncn(C2OC(COP(=O)(O)O)C(O)C2O)c1N,7.258918
1,1004,ACH-002137,CCC1(O)CC2CN(CCc3c([nH]c4ccccc34)C(C(=O)OC)(c3...,-3.802467
2,1005,ACH-002137,N.N.[Cl-].[Cl-].[Pt+2],4.146364
3,1006,ACH-002137,Nc1ccn(C2OC(CO)C(O)C2O)c(=O)n1,3.171367
4,1007,ACH-002137,CC(=O)OC12COC1CC(O)C1(C)C(=O)C(O)C3=C(C)C(OC(=...,-4.959442


In [34]:
drugs_smiles_cell_lines_ic50_df.shape

(186789, 4)

In [35]:
ttt = drugs_smiles_cell_lines_ic50_df.drop(["IC50"],axis = 1)

In [36]:
x_train, x_valid, y_train, y_valid = train_test_split(drugs_smiles_cell_lines_ic50_df.drop(["IC50"],axis = 1), drugs_smiles_cell_lines_ic50_df["IC50"].values, 
                                                     test_size = 0.20, random_state = 42)

In [37]:
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((149431, 3), (37358, 3), (149431,), (37358,))

In [38]:
dualgcn_train["Drug_ID"] = dualgcn_train["Drug_ID"].astype(str)

In [39]:
dualgcn_test["Drug_ID"] = dualgcn_test["Drug_ID"].astype(str)

In [40]:
pubchem_to_drugs_df.dtypes

drug_id    object
PubCHEM    object
dtype: object

In [41]:
# dualgcn_train

In [42]:
dualgcn_train = pubchem_to_drugs_df.merge(dualgcn_train, left_on = ["PubCHEM"], right_on = ["Drug_ID"])

In [43]:
dualgcn_train = dualgcn_train[['Cell_Line', 'drug_id']]

In [44]:
dualgcn_train.shape

(69214, 2)

In [45]:
dualgcn_train.head()

,Cell_Line,drug_id
0,ACH-000070,1242
1,ACH-000105,1242
2,ACH-000981,1242
3,ACH-000061,1242
4,ACH-000995,1242


In [46]:
print(dualgcn_train['Cell_Line'].value_counts().size)
print(dualgcn_train['drug_id'].value_counts().size)

525
208


In [47]:
dualgcn_test = pubchem_to_drugs_df.merge(dualgcn_test, left_on = ["PubCHEM"], right_on = ["Drug_ID"])

In [48]:
dualgcn_test = dualgcn_test[['Cell_Line', 'drug_id']]

In [49]:
dualgcn_train.dtypes

Cell_Line    object
drug_id      object
dtype: object

In [50]:
x_train.dtypes

drug_id             object
Cancer_Cell_Line    object
Smiles              object
dtype: object

In [51]:
x_train['drug_id'] = x_train['drug_id'].astype(str)

In [52]:
x_valid['drug_id'] = x_valid['drug_id'].astype(str)

In [53]:
print(x_train.dtypes, x_valid.dtypes)

drug_id             object
Cancer_Cell_Line    object
Smiles              object
dtype: object drug_id             object
Cancer_Cell_Line    object
Smiles              object
dtype: object


In [54]:
x_train_valid_feats = pd.concat([x_train, x_valid], ignore_index = True)

In [55]:
y_train_valid = pd.concat([pd.DataFrame(y_train.reshape(-1,1)), pd.DataFrame(y_valid.reshape(-1,1))], ignore_index = True)

In [56]:
x_train_only = x_train_valid_feats.iloc[:149431, :]
x_valid_only = x_train_valid_feats.iloc[149431:, :]

In [57]:
# quick verifying of the train and valid data is concatenated correctly
print(np.mean(x_train_only['drug_id'].values == x_train['drug_id'].values))
print(np.mean(x_train_only['Cancer_Cell_Line'].values == x_train['Cancer_Cell_Line'].values))
print(np.mean(x_train_only['Smiles'].values == x_train['Smiles'].values))

print(np.mean(x_valid_only['drug_id'].values == x_valid['drug_id'].values))
print(np.mean(x_valid_only['Cancer_Cell_Line'].values == x_valid['Cancer_Cell_Line'].values))
print(np.mean(x_valid_only['Smiles'].values == x_valid['Smiles'].values))

1.0
1.0
1.0
1.0
1.0
1.0


In [58]:
# quick verifying of the train and valid data is concatenated correctly
y_train_only = y_train_valid.iloc[:149431, :]
y_valid_only = y_train_valid.iloc[149431:, :]

In [59]:
print(np.mean(y_train_only.values.reshape(1,-1) == y_train))
print(np.mean(y_valid_only.values.reshape(1,-1) == y_valid))

1.0
1.0


In [60]:
np.mean(y_train), np.mean(y_valid), np.std(y_train), np.std(y_valid)

(2.07713533482343, 2.0784021872423577, 2.723665295435388, 2.716344975603266)

In [61]:
combo_train_valid = pd.concat([x_train_valid_feats, y_train_valid], axis = 1)

In [62]:
# combo_train_valid

In [63]:
combo_train_valid.columns = ['drug_id', 'Cancer_Cell_Line', 'Smiles', 'IC50']

In [64]:
combo_train_valid.head()

,drug_id,Cancer_Cell_Line,Smiles,IC50
0,1030,ACH-000295,O=c1cc(-c2cccc3c2Sc2ccccc2S3)oc(N2CCOCC2)c1,3.143044
1,203,ACH-000292,Cc1ccc2nc(NCCN)c3ncc(C)n3c2c1,2.772620
2,1072,ACH-002273,NC(=O)C(CCC(F)(F)F)N(Cc1ccc(-c2ncon2)cc1F)S(=O...,3.809977
3,150,ACH-000631,CC(O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C#N)c(...,3.053030
4,282,ACH-000270,CCOc1cc2ncc(C#N)c(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)...,-0.367467


In [65]:
combo_train_valid.shape

(186789, 4)

In [66]:
# filter x_train x _valid here
x_y_train = combo_train_valid.merge(dualgcn_train, left_on = ['Cancer_Cell_Line','drug_id'], right_on = [ 'Cell_Line','drug_id'])

In [67]:
x_y_test = combo_train_valid.merge(dualgcn_test, left_on = ['Cancer_Cell_Line','drug_id'], right_on = [ 'Cell_Line','drug_id'])

In [68]:
x_y_train.head()

,drug_id,Cancer_Cell_Line,Smiles,IC50,Cell_Line
0,203,ACH-000292,Cc1ccc2nc(NCCN)c3ncc(C)n3c2c1,2.772620,ACH-000292
1,282,ACH-000270,CCOc1cc2ncc(C#N)c(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)...,-0.367467,ACH-000270
2,51,ACH-000087,Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(N2CCN(C...,2.620152,ACH-000087
3,167,ACH-000948,NCC(=O)Nc1ccc(-n2nc(C(F)(F)F)cc2-c2ccc3c(ccc4c...,2.061684,ACH-000948
4,180,ACH-000971,CC=C(C)C(=O)OC1C(C)=C2C(C1OC(=O)CCCCCCC)C(C)(O...,-5.105745,ACH-000971


In [69]:
x_y_train.shape

(69214, 5)

In [70]:
np.mean(x_y_train['IC50']), np.std(x_y_train['IC50'])

(2.049294179775768, 2.8425066110138553)

In [71]:
x_y_test.head()

,drug_id,Cancer_Cell_Line,Smiles,IC50,Cell_Line
0,1030,ACH-000295,O=c1cc(-c2cccc3c2Sc2ccccc2S3)oc(N2CCOCC2)c1,3.143044,ACH-000295
1,249,ACH-000613,COc1cc2nccc(Oc3ccc(NC(=O)C4(C(=O)Nc5ccc(F)cc5)...,3.213836,ACH-000613
2,1005,ACH-000802,N.N.[Cl-].[Cl-].[Pt+2],1.036346,ACH-000802
3,300,ACH-000174,Cc1cnc(CNC(=O)c2c(=O)c3ccc(N4CCCN(C)CC4)nc3n3c...,3.379955,ACH-000174
4,178,ACH-000362,COc1ccc(-c2cc3nccn3c(Nc3ncccc3C(N)=O)n2)cc1OC,-2.087380,ACH-000362


In [72]:
x_y_test.shape

(17316, 5)

In [73]:
dualgcn_train['Cell_Line'].value_counts()

Cell_Line
ACH-000647    179
ACH-000800    179
ACH-000151    177
ACH-000157    176
ACH-000295    176
             ... 
ACH-000763     41
ACH-000727     35
ACH-000345     24
ACH-000225     20
ACH-000035     15
Name: count, Length: 525, dtype: int64

In [74]:
x_y_train['Cancer_Cell_Line'].value_counts()

Cancer_Cell_Line
ACH-000647    179
ACH-000800    179
ACH-000151    177
ACH-000295    176
ACH-000157    176
             ... 
ACH-000763     41
ACH-000727     35
ACH-000345     24
ACH-000225     20
ACH-000035     15
Name: count, Length: 525, dtype: int64

In [75]:
dualgcn_test['Cell_Line'].value_counts()

Cell_Line
ACH-000488    53
ACH-000706    53
ACH-000983    52
ACH-000943    52
ACH-000353    52
              ..
ACH-000506     8
ACH-000681     8
ACH-000187     6
ACH-000225     4
ACH-000035     3
Name: count, Length: 524, dtype: int64

In [76]:
x_y_test['Cancer_Cell_Line'].value_counts()

Cancer_Cell_Line
ACH-000488    53
ACH-000706    53
ACH-000943    52
ACH-000983    52
ACH-000353    52
              ..
ACH-000506     8
ACH-000681     8
ACH-000187     6
ACH-000225     4
ACH-000035     3
Name: count, Length: 524, dtype: int64

In [77]:
# Do a train test split here for the train and validation sets based on the train dataset - have a test size of 20%

In [78]:
x_train, x_valid, y_train, y_valid = train_test_split(x_y_train.drop(["IC50", 'Cell_Line'],axis = 1),x_y_train["IC50"].values, random_state = 42, test_size = 0.2)

In [79]:
print("x train shape", x_train.shape)
print("x valid shape", x_valid.shape)
print("y train shape", y_train.shape)
print("y valid shape", y_valid.shape)

x train shape (55371, 3)
x valid shape (13843, 3)
y train shape (55371,)
y valid shape (13843,)


In [80]:
np.mean(y_train), np.std(y_train), np.mean(y_valid), np.std(y_valid)

(2.0558525880515073, 2.8441288895772825, 2.0230610204435453, 2.835856668835907)

In [81]:
x_test, y_test = x_y_test.drop(["IC50", 'Cell_Line'], axis = 1), x_y_test["IC50"].values

In [82]:
print("x test shape", x_test.shape)
print("y test shape", y_test.shape)

x test shape (17316, 3)
y test shape (17316,)


In [83]:
np.mean(y_test), np.std(y_test)

(2.063824293081543, 2.8309603843576316)

In [84]:
train_gcn_feats = []
train_adj_list = []
for drug_id in x_train["drug_id"].values:
    train_gcn_feats.append(dict_features[drug_id])
    train_adj_list.append(dict_normalized_adj_mats[drug_id])

In [85]:
valid_gcn_feats = []
valid_adj_list = []
for drug_id in x_valid["drug_id"].values:
    valid_gcn_feats.append(dict_features[drug_id])
    valid_adj_list.append(dict_normalized_adj_mats[drug_id])

In [86]:
test_gcn_feats = []
test_adj_list = []
for drug_id in x_test["drug_id"].values:
    test_gcn_feats.append(dict_features[drug_id])
    test_adj_list.append(dict_normalized_adj_mats[drug_id])

In [87]:
%%time
train_gcn_feats = np.array(train_gcn_feats).astype("float16")
valid_gcn_feats = np.array(valid_gcn_feats).astype("float16")
test_gcn_feats = np.array(test_gcn_feats).astype("float16")

CPU times: user 2.69 s, sys: 698 ms, total: 3.39 s
Wall time: 3.4 s


In [88]:
print(np.isnan(train_gcn_feats).any())
print(np.isnan(valid_gcn_feats).any())
print(np.isnan(test_gcn_feats).any())

False
False
False


In [89]:
%%time
train_adj_list = np.array(train_adj_list).astype("float16")
valid_adj_list = np.array(valid_adj_list).astype("float16")
test_adj_list = np.array(test_adj_list).astype("float16")

CPU times: user 3.6 s, sys: 909 ms, total: 4.51 s
Wall time: 4.51 s


In [90]:
print(np.isnan(train_adj_list).any())
print(np.isnan(valid_adj_list).any())
print(np.isnan(test_adj_list).any())

False
False
False


In [91]:
# load models
# omic models
cancer_copy_number_model = tf.keras.models.load_model("..//models//cancer_copy_number_model_no_norm_common")
cancer_cell_gen_expr_model = tf.keras.models.load_model("..//models//cancer_cell_gen_expr_model_no_norm_common")
cancer_cell_gen_methy_model = tf.keras.models.load_model("..//models//cancer_cell_gen_methy_model_no_norm")
cancer_cell_gen_mut_model = tf.keras.models.load_model("..//models//cancer_cell_gen_mut_model_no_norm")

2024-07-11 15:55:47.125568: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


In [92]:
# load models
# drug models
pubchem_drugs_rdkit_model = tf.keras.models.load_model("..//models//pubchem_drugs_rdkit_model_no_norm")

In [93]:
std = StandardScaler()

In [94]:
# extract drug features
drug_features_train = pubchem_drugs_rdkit_model(x_train["drug_id"].values).numpy().astype("float32")
drug_features_valid = pubchem_drugs_rdkit_model(x_valid["drug_id"].values).numpy().astype("float32")
drug_features_test = pubchem_drugs_rdkit_model(x_test["drug_id"].values).numpy().astype("float32")

In [95]:
drug_features_train.shape, drug_features_valid.shape, drug_features_test.shape

((55371, 106), (13843, 106), (17316, 106))

In [96]:
np.isinf(drug_features_train).sum()

0

In [97]:
drug_features_train = std.fit_transform(drug_features_train)

In [98]:
drug_features_valid = std.transform(drug_features_valid)
drug_features_test = std.transform(drug_features_test)

In [99]:
print(np.isnan(drug_features_train).any())
print(np.isnan(drug_features_valid).any())
print(np.isnan(drug_features_test).any())

False
False
False


In [100]:
# extract copy number features
omics_copy_number_train = cancer_copy_number_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_copy_number_valid = cancer_copy_number_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_copy_number_test = cancer_copy_number_model(x_test["Cancer_Cell_Line"].values).numpy().astype("float16")

In [101]:
omics_copy_number_train.shape, omics_copy_number_valid.shape, omics_copy_number_test.shape

((55371, 691), (13843, 691), (17316, 691))

In [102]:
print(np.isnan(omics_copy_number_train).any())
print(np.isnan(omics_copy_number_valid).any())
print(np.isnan(omics_copy_number_test).any())

False
False
False


In [103]:
# extract gen expr features
omics_gen_expr_train = cancer_cell_gen_expr_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_gen_expr_valid = cancer_cell_gen_expr_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_gen_expr_test = cancer_cell_gen_expr_model(x_test["Cancer_Cell_Line"].values).numpy().astype("float16")

In [104]:
omics_gen_expr_train.shape, omics_gen_expr_valid.shape, omics_gen_expr_test.shape

((55371, 691), (13843, 691), (17316, 691))

In [105]:
print(np.isnan(omics_gen_expr_train).any())
print(np.isnan(omics_gen_expr_valid).any())
print(np.isnan(omics_gen_expr_test).any())

False
False
False


In [106]:
# extract gen methylation features
omics_gen_methyl_train = cancer_cell_gen_methy_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_gen_methyl_valid = cancer_cell_gen_methy_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_gen_methyl_test= cancer_cell_gen_methy_model(x_test["Cancer_Cell_Line"].values).numpy().astype("float16")

In [107]:
omics_gen_methyl_train.shape, omics_gen_methyl_valid.shape, omics_gen_methyl_test.shape

((55371, 808), (13843, 808), (17316, 808))

In [108]:
print(np.isnan(omics_gen_methyl_train).any())
print(np.isnan(omics_gen_methyl_valid).any())
print(np.isnan(omics_gen_methyl_test).any())

False
False
False


In [109]:
# extract gen mutation features
with tf.device('/cpu:0'):
    omics_gen_mut_train = cancer_cell_gen_mut_model.predict(x_train["Cancer_Cell_Line"].values, verbose = 1, batch_size = 256).astype("float16")
    omics_gen_mut_valid = cancer_cell_gen_mut_model.predict(x_valid["Cancer_Cell_Line"].values, verbose = 1, batch_size = 256).astype("float16")
    omics_gen_mut_test = cancer_cell_gen_mut_model.predict(x_test["Cancer_Cell_Line"].values, verbose = 1, batch_size = 256).astype("float16")

68/68 [==============================] - 0s 5ms/step


In [110]:
omics_gen_mut_train.shape, omics_gen_mut_valid.shape, omics_gen_mut_test.shape

((55371, 34673), (13843, 34673), (17316, 34673))

In [111]:
print(np.isnan(omics_gen_mut_train).any())
print(np.isnan(omics_gen_mut_valid).any())
print(np.isnan(omics_gen_mut_test).any())

False
False
False


In [112]:
smile_strings_train = x_train["Smiles"].values.reshape(-1,1)
smile_strings_valid = x_valid["Smiles"].values.reshape(-1,1)
smile_strings_test = x_test["Smiles"].values.reshape(-1,1)

In [113]:
smile_strings_train.shape, smile_strings_valid.shape, smile_strings_test.shape

((55371, 1), (13843, 1), (17316, 1))

In [114]:
input_gcn_features = tf.keras.layers.Input(shape = (100, 75))
input_norm_adj_mat = tf.keras.layers.Input(shape = (100, 100))
mult_1 = tf.keras.layers.Dot(1)([input_norm_adj_mat, input_gcn_features])
dense_layer_gcn = tf.keras.layers.Dense(256, activation = "relu")
dense_out = dense_layer_gcn(mult_1)
dense_out = tf.keras.layers.BatchNormalization()(dense_out)
dense_out = tf.keras.layers.Dropout(0.2)(dense_out)
mult_2 = tf.keras.layers.Dot(1)([input_norm_adj_mat, dense_out])
dense_layer_gcn = tf.keras.layers.Dense(256, activation = "relu")
dense_out = dense_layer_gcn(mult_2)
dense_out = tf.keras.layers.BatchNormalization()(dense_out)
dense_out = tf.keras.layers.Dropout(0.2)(dense_out)

dense_layer_gcn = tf.keras.layers.Dense(100, activation = "relu")
mult_3 = tf.keras.layers.Dot(1)([input_norm_adj_mat, dense_out])
dense_out = dense_layer_gcn(mult_3)
dense_out = tf.keras.layers.BatchNormalization()(dense_out)
dense_out = tf.keras.layers.Dropout(0.2)(dense_out)

dense_out = tf.keras.layers.GlobalAvgPool1D()(dense_out)

In [115]:
input_gen_expr = tf.keras.layers.Input(shape = (omics_gen_expr_train.shape[1]))

In [116]:
l1 = tf.keras.layers.Dense(256, activation = "tanh")(input_gen_expr)
l1 = tf.keras.layers.BatchNormalization()(l1)
l1 = tf.keras.layers.Dropout(0.2)(l1)
l1 = tf.keras.layers.Dense(100, activation = "relu")(l1)
# l2 = tf.keras.layers.Dense(128)(l1)
# l2 = tf.keras.layers.Dropout(0.2)(l2)
# mult_11 = tf.keras.layers.Dot(1)([const_input, l2])


In [117]:
input_gen_methy = tf.keras.layers.Input(shape = (omics_gen_methyl_train.shape[1],))

In [118]:
gen_methy_layer = tf.keras.layers.Dense(256, activation = "tanh")

In [119]:
gen_methy_emb = gen_methy_layer(input_gen_methy)
gen_methy_emb = tf.keras.layers.BatchNormalization()(gen_methy_emb)
gen_methy_emb = tf.keras.layers.Dropout(0.2)(gen_methy_emb)

In [120]:
gen_methy_layer = tf.keras.layers.Dense(100, activation = "relu")
gen_methy_emb = gen_methy_layer(gen_methy_emb)

In [121]:
input_gen_mut = tf.keras.layers.Input(shape = (omics_gen_mut_train.shape[1],))

In [122]:
reshape_gen_mut = tf.keras.layers.Reshape((1, omics_gen_mut_train.shape[1], 1))

In [123]:
reshape_gen_mut = reshape_gen_mut(input_gen_mut)

In [124]:
gen_mut_layer = tf.keras.layers.Conv2D(50, (1, 700), strides=5, activation = "tanh")

In [125]:
gen_mut_emb = gen_mut_layer(reshape_gen_mut)

In [126]:
pool_layer = tf.keras.layers.MaxPooling2D((1,5))

In [127]:
pool_out = pool_layer(gen_mut_emb)

In [128]:
gen_mut_layer = tf.keras.layers.Conv2D(30, (1, 5), strides=2, activation = "relu")

In [129]:
gen_mut_emb = gen_mut_layer(pool_out)

In [130]:
pool_layer = tf.keras.layers.MaxPooling2D((1,10))

In [131]:
pool_out = pool_layer(gen_mut_emb)

In [132]:
flatten_layer = tf.keras.layers.Flatten()

In [133]:
flatten_out = flatten_layer(pool_out)

In [134]:
all_omics = tf.keras.layers.Concatenate()([gen_methy_emb, flatten_out, l1, dense_out])

In [135]:
x = tf.keras.layers.Dense(300,activation = 'tanh')(all_omics)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Lambda(lambda x: K.expand_dims(x,axis=-1))(x)
x = tf.keras.layers.Lambda(lambda x: K.expand_dims(x,axis=1))(x)
x = tf.keras.layers.Conv2D(filters=30, kernel_size=(1,150),strides=(1, 1), activation = 'relu',padding='valid')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(1,2))(x)
x = tf.keras.layers.Conv2D(filters=10, kernel_size=(1,5),strides=(1, 1), activation = 'relu',padding='valid')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(1,3))(x)
x = tf.keras.layers.Conv2D(filters=5, kernel_size=(1,5),strides=(1, 1), activation = 'relu',padding='valid')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(1,3))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)

In [136]:
final_out_layer = tf.keras.layers.Dense(1)

In [137]:
final_out = final_out_layer(x)

In [138]:
simplecdr = tf.keras.models.Model([input_gcn_features, input_norm_adj_mat,input_gen_expr,
                                   input_gen_methy, input_gen_mut], final_out)

In [139]:
simplecdr.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 100, 100)]           0         []                            
                                                                                                  
 input_1 (InputLayer)        [(None, 100, 75)]            0         []                            
                                                                                                  
 dot (Dot)                   (None, 100, 75)              0         ['input_2[0][0]',             
                                                                     'input_1[0][0]']             
                                                                                                  
 dense (Dense)               (None, 100, 256)             19456     ['dot[0][0]']             

In [140]:
simplecdr.compile(loss = tf.keras.losses.MeanSquaredError(), 
                    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
                    metrics = [tf.keras.metrics.RootMeanSquaredError()])

In [141]:
# !conda install graphviz

In [142]:
# tf.keras.utils.plot_model(simplecdr)

In [143]:
print(np.isnan(y_train).any())
print(np.isnan(y_valid).any())
print(np.isnan(y_test).any())

False
False
False


In [144]:
%%time
history = simplecdr.fit([train_gcn_feats, train_adj_list,
                         omics_gen_expr_train, 
                         omics_gen_methyl_train, omics_gen_mut_train], y_train, 
                         
          batch_size = 512, epochs = 1000, verbose = 1,
                         
          validation_data=([ valid_gcn_feats, valid_adj_list,
                           omics_gen_expr_valid, 
                           omics_gen_methyl_valid, omics_gen_mut_valid], y_valid),
                         

        callbacks = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 20, restore_best_weights=True,
                                                       mode = "min"), 
         validation_batch_size = 512, shuffle = True)

Epoch 1/1000


2024-07-11 15:56:43.725267: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout_6/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-07-11 15:56:44.396020: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-07-11 15:56:46.110869: I external/local_xla/xla/service/service.cc:168] XLA service 0x1533e4d04310 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-11 15:56:46.110905: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2024-07-11 15:56:46.154293: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1720731406.437935   54624 device_compiler.h:186] Compiled cluster using XLA!  T

109/109 [==============================] - 32s 194ms/step - loss: 5.6887 - root_mean_squared_error: 2.3851 - val_loss: 8.5493 - val_root_mean_squared_error: 2.9239
Epoch 2/1000
109/109 [==============================] - 16s 144ms/step - loss: 3.1622 - root_mean_squared_error: 1.7783 - val_loss: 8.0174 - val_root_mean_squared_error: 2.8315
Epoch 3/1000
109/109 [==============================] - 16s 145ms/step - loss: 2.6521 - root_mean_squared_error: 1.6285 - val_loss: 7.6012 - val_root_mean_squared_error: 2.7570
Epoch 4/1000
109/109 [==============================] - 16s 145ms/step - loss: 2.4653 - root_mean_squared_error: 1.5701 - val_loss: 6.7591 - val_root_mean_squared_error: 2.5998
Epoch 5/1000
109/109 [==============================] - 16s 145ms/step - loss: 2.3378 - root_mean_squared_error: 1.5290 - val_loss: 5.3452 - val_root_mean_squared_error: 2.3120
Epoch 6/1000
109/109 [==============================] - 16s 145ms/step - loss: 2.2599 - root_mean_squared_error: 1.5033 - val_lo

In [145]:
simplecdr.save("..//models//deepcdr_trained_on_domain")

INFO:tensorflow:Assets written to: ..//models//deepcdr_trained_on_domain/assets


INFO:tensorflow:Assets written to: ..//models//deepcdr_trained_on_domain/assets


In [146]:
# !sudo apt install graphviz

In [147]:
# tf.keras.utils.plot_model(simplecdr)